# Implementing Basis Functions for Gillan's Method

[1] https://doi.org/10.1080/00268977900102861

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=2, suppress=True)

Define a grid:

In [ ]:
radius = 20.48
npts = 1024
dx = radius/npts
x = np.arange(0.5, npts) * dx

Defines the number of basis functions and their corresponding nodes:

In [ ]:
nbasis = 6
node = int(npts / nbasis / 2)
print(node)
nodes_index = np.arange(0, nbasis+1) * node
nodes = x[nodes_index]
print(nodes_index)
print(x)
print(nodes)

Initialise the Basis functions and related values:

In [ ]:
P = np.zeros((npts, nbasis+1), dtype=np.float64) # Basis Function Eq(17) of [1]
Q = np.zeros_like(P) # Conjugate Basis Function Eq(18) of [1]
R = np.zeros((nbasis, nbasis), dtype=np.float64) # Eq (20) of [1]
kron = np.zeros_like(R) # Eq (19) of [1]

Basis function $P$ is defined with the following cases: 

(1) $P_\alpha^i = 0$ when $1 \leq i \leq i_{\alpha-2}$

(2) $P_\alpha^i = \frac{i - i_{\alpha-2}}{i_{\alpha-1} - i_{\alpha-2}}$ when $i_{\alpha-2} \leq i \leq i_{\alpha-1}$

(3) $P_\alpha^i = \frac{i_\alpha - i}{i_{\alpha} - i_{\alpha-1}}$ when $i_{\alpha-1} \leq i \leq i_{\alpha}$

(4) $P_\alpha^i = 0$ when $i_{\alpha} \leq i \leq N$

and for the case $P_1^i$ ($\alpha = 1$):

(5) $P_1^i = \frac{i_1 - i}{i_1}$ when $1 \leq i \leq i_1$

(6) $P_1^i = 0$ when $i_1 \leq i \leq N$

where $i_\alpha$ are the points at which a node is defined (var: nodes).
Since the arrays are initialised with 0, we only really need to implement (2), (3) and (5).

This paper was written most likely for Fortran where arrays start at 1, but in Python arrays are 0-indexed! Keep that in mind.

In [ ]:
# For the case \alpha = 0 over all idx
P[...,0] = 1
# For \alpha = 1 and the rest of the idx
for idx in range(0, npts):
    if nodes[0] <= x[idx] <= nodes[1]:
        P[idx, 1] = (nodes[1] - x[idx]) / (nodes[1])

In [ ]:
plt.plot(x, P[..., 1])
plt.show()

In [ ]:
# Rest of the \alpha cases
for a in range(2, nbasis+1):
    for idx in range(0, npts):
        if nodes[a-2] <= x[idx] <= nodes[a-1]:
            P[idx, a] = (x[idx] - nodes[a-2])/(nodes[a-1] - nodes[a-2])
        elif nodes[a-1] <= x[idx] <= nodes[a]:
            P[idx, a] = (nodes[a] - x[idx])/(nodes[a] - nodes[a-1])

In [ ]:
for a in range(1, nbasis+1):
    print(P[..., a])
    plt.axvline(x=nodes[a], color='grey', linestyle="--", linewidth=0.5)
    plt.plot(x, P[..., a])
plt.xlim([0.0, nodes[-1]+2])
plt.show()

Implementing equation 20 of [1] - $R_{\alpha\beta} = \sum_i P_\alpha^i P_\beta^i$

In [ ]:
P_skip_zero = P[..., 1:].copy()
Pa = P_skip_zero.copy()
Pb = P_skip_zero.copy()
for a in range(nbasis):
    for b in range(nbasis):
        R[a,b] = (Pa[..., a] * Pb[..., b]).sum()

The inverse of R - $B_{\alpha\beta}$

In [ ]:
B = np.linalg.inv(R)

Implementing equation 18 of [1] - $Q_\alpha^i = \sum_\beta B_{\alpha\beta}P_\beta^i$

In [ ]:
for i in range(npts):
    for a in range(nbasis):
        Q[i,a] = (B[a,:] * P_skip_zero[i,:]).sum()

Implementing equation 20 of [1] - $\sum_i Q_\alpha^iP_\beta^i = \delta_{\alpha\beta}$

In [ ]:
for a in range(nbasis):
    for b in range(nbasis):
        kron[a,b] = (Q[...,a] * P_skip_zero[...,b]).sum()

kron should be the identity matrix...

In [ ]:
print(kron)